# Data Preparation (clean version)

In [1]:
import pandas as pd
import numpy as np
from pandas.io import sql
import sqlite3

In [2]:
conn = sqlite3.connect('../assets/wmata.db')

In [3]:
# Read in volume data by day of week, quarter-hour, and entry and exit station

vol_data = pd.read_csv("../assets/2015-10-OD.csv", header=0)
print(vol_data.head())
print(vol_data.info())

  ENTDATESERVICETYPE ENTDATEDAYOFWEEK      ENTSTATION  \
0           Saturday              Sat       Navy Yard   
1            Weekday              Thu      Huntington   
2            Weekday              Thu     Shady Grove   
3            Weekday              Thu    Foggy Bottom   
4            Weekday              Thu  L'Enfant Plaza   

                EXTSTATION ENTPERIOD        ENTQUARTHOUR  AVG_TRIPS  
0            Dupont Circle   PM Peak  4:00 PM to 4:15 PM          2  
1  Gallery Place-Chinatown   AM Peak  5:30 AM to 5:45 AM          5  
2         Judiciary Square   AM Peak  9:00 AM to 9:15 AM         11  
3          Van Dorn Street   PM Peak  5:45 PM to 6:00 PM         10  
4         West Hyattsville   PM Peak  5:30 PM to 5:45 PM         12  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2301605 entries, 0 to 2301604
Data columns (total 7 columns):
ENTDATESERVICETYPE    object
ENTDATEDAYOFWEEK      object
ENTSTATION            object
EXTSTATION            object
ENTPERIOD

In [4]:
print(vol_data.ENTSTATION.unique())
len(vol_data.ENTSTATION.unique())

['Navy Yard' 'Huntington' 'Shady Grove' 'Foggy Bottom' "L'Enfant Plaza"
 'Federal Center SW' 'Capitol Heights' 'West Falls Church' 'Union Station'
 'Archives-Navy Memorial' 'Metro Center' 'Gallery Place-Chinatown'
 'McPherson Square' 'Dunn Loring' 'Bethesda' 'Southern Avenue' 'Rosslyn'
 'Columbia Heights' 'Friendship Heights' 'Dupont Circle' 'Farragut North'
 'Takoma' 'King Street' 'New Carrollton' 'Cleveland Park' 'Brookland'
 'College Park-U of MD' 'Judiciary Square' 'Tenleytown-AU' 'Benning Road'
 'Potomac Avenue' 'Landover' 'Spring Hill' 'Farragut West' 'Medical Center'
 'Georgia Avenue-Petworth' 'Court House' 'Mt. Vernon Square-UDC'
 'Addison Road' 'Fort Totten' 'Greenbelt' 'U Street-Cardozo'
 'Braddock Road' 'Pentagon' 'Crystal City' 'Federal Triangle' 'Smithsonian'
 'Anacostia' 'Silver Spring' 'Eisenhower Avenue'
 'Reagan Washington National Airport' 'Virginia Square-GMU' 'Pentagon City'
 'Suitland' "Prince George's Plaza" 'Eastern Market' 'Twinbrook'
 'White Flint' 'Naylor Road

91

In [5]:
# Read in station and line length data

stn_data = pd.read_excel("../assets/Rail-Passenger-Link-Volumes-by-Line-by-Hour-May-2015-weekday.xlsx", header=None,
                        names=["Stop", "Length", "Alight", "Board", "Volume", "CarsPerHour", "PassPerCar", "StopName", "Hour", "SvcPattern", "LinkCode"],
                        parse_cols="A:E,H:M")

In [6]:
stn_data.head()

,Stop,Length,Alight,Board,Volume,CarsPerHour,PassPerCar,StopName,Hour,SvcPattern,LinkCode
0,Stop,Length,Alight,Board,Hourly Link Volume (Riders on Board),Scheduled Cars per Hour,Estimated Passengers per Car,StopName,HOUR,Service Pattern,LINK_CODE (first stop - track - second stop)
1,1,4:00..5:00,0,Red Lines,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,5:00..6:00,54,Red Lines,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,13722,0,364,364,26,14,Shady Grove -- EB,5:00..6:00,Red,A15-1-A14


In [7]:
# Force the first seven columns to numeric, coercing non-numeric values to NaN

pd.to_numeric(stn_data['Stop'], errors='coerce')
pd.to_numeric(stn_data['Length'], errors='coerce')
pd.to_numeric(stn_data['Alight'], errors='coerce')
pd.to_numeric(stn_data['Board'], errors='coerce')
pd.to_numeric(stn_data['Volume'], errors='coerce')
pd.to_numeric(stn_data['CarsPerHour'], errors='coerce')
pd.to_numeric(stn_data['PassPerCar'], errors='coerce')

0        NaN
1        NaN
2        NaN
3        NaN
4       14.0
5       15.7
6       17.3
7       18.2
8       20.3
9       19.0
10      19.6
11      20.3
12      20.5
13      21.3
14      21.8
15      22.5
16      22.7
17      20.4
18      20.7
19      21.8
20      18.9
21      11.0
22       8.3
23       7.7
24       6.9
25       7.5
26       7.4
27       3.2
28       2.5
29       1.1
        ... 
5886    27.1
5887    24.9
5888    19.7
5889    15.6
5890    15.2
5891    13.5
5892    10.2
5893     9.3
5894     0.0
5895     NaN
5896     0.2
5897     0.2
5898     0.8
5899     1.5
5900     1.7
5901     2.0
5902     4.3
5903     4.7
5904     5.7
5905     5.5
5906     3.7
5907     3.0
5908     3.3
5909     2.7
5910     2.3
5911     2.5
5912     0.0
5913     NaN
5914     NaN
5915     NaN
Name: PassPerCar, Length: 5916, dtype: float64

In [8]:
# Drop rows with missing values

print(len(stn_data))
stn_data.dropna(axis='index', inplace=True)
print(len(stn_data))

5916
5751


In [9]:
# Drop header row

stn_data = stn_data[1:]
print(len(stn_data))

5750


In [10]:
# Derive station name from StopName

stn_data['StationName'] = stn_data.StopName.apply(func=(lambda x: x[0:(len(x)-6)]))
stn_data.head()

,Stop,Length,Alight,Board,Volume,CarsPerHour,PassPerCar,StopName,Hour,SvcPattern,LinkCode,StationName
4,1,13722,0,364,364,26,14,Shady Grove -- EB,5:00..6:00,Red,A15-1-A14,Shady Grove
5,2,11260,3,46,407,26,15.7,Rockville -- EB,5:00..6:00,Red,A14-1-A13,Rockville
6,3,5678,7,50,450,26,17.3,Twinbrook -- EB,5:00..6:00,Red,A13-1-A12,Twinbrook
7,4,7222,12,36,474,26,18.2,White Flint -- EB,5:00..6:00,Red,A12-1-A11,White Flint
8,5,11091,2,57,529,26,20.3,Grosvenor-Strathmore -- EB,5:00..6:00,Red,A11-1-A10,Grosvenor-Strathmore


In [11]:
# Rename StationName to StnName1

stn_data=stn_data.rename(columns = {'StationName':'StnName1'})

In [12]:
# Since we deleted rows, reindex the data

stn_data.index = np.arange(0, len(stn_data))

In [13]:
# The second station in each link is the station name from the following line. Make this a Series

StnName2 = stn_data.loc[1:, 'StnName1']

In [14]:
# Reindex StnName2, then assign it to a new column in stn_data

StnName2.index = np.arange(0, len(StnName2))
stn_data['StnName2'] = StnName2

In [15]:
# Derive station codes and directions from LinkCode and StopName

stn_data['StnCode1'] = stn_data.LinkCode.str[0:3]
stn_data['StnCode2'] = stn_data.LinkCode.str[6:9]
stn_data['Direction'] = stn_data.StopName.str[-2:]

In [16]:
# Select certain columns of stn_data into a new data frame, link_lengths

link_lengths = pd.DataFrame(data=[stn_data['StnName1'], stn_data['StnName2'], stn_data['StnCode1'], stn_data['StnCode2'], stn_data['Direction'], stn_data['Length'], stn_data['SvcPattern']])
link_lengths = link_lengths.transpose()

In [17]:
# Keep only unique rows in link_lengths

link_lengths.drop_duplicates(inplace=True)
len(link_lengths)

324

In [18]:
# Drop rows where the link length is zero; these are arrivals at terminal stations

link_lengths = link_lengths[link_lengths.Length != 0]
len(link_lengths)

293

In [19]:
# Rename "SvcPattern" to "Line," since it is simply the line color

link_lengths=link_lengths.rename(columns = {'SvcPattern':'Line'})
link_lengths

,StnName1,StnName2,StnCode1,StnCode2,Direction,Length,Line
0,Shady Grove,Rockville,A15,A14,EB,13722,Red
1,Rockville,Twinbrook,A14,A13,EB,11260,Red
2,Twinbrook,White Flint,A13,A12,EB,5678,Red
3,White Flint,Grosvenor-Strathmore,A12,A11,EB,7222,Red
4,Grosvenor-Strathmore,Medical Center,A11,A10,EB,11091,Red
5,Medical Center,Bethesda,A10,A09,EB,5764,Red
6,Bethesda,Friendship Heights,A09,A08,EB,9485,Red
7,Friendship Heights,Tenleytown-AU,A08,A07,EB,4167,Red
8,Tenleytown-AU,Van Ness-UDC,A07,A06,EB,5734,Red
9,Van Ness-UDC,Cleveland Park,A06,A05,EB,2886,Red


In [20]:
# Verify which directions exist in the data

link_lengths.Direction.unique()

array(['EB', 'WB', 'NB', 'SB'], dtype=object)

In [21]:
# Verify which directions occur with which lines

link_lengths.Line.add(link_lengths.Direction).unique()

# NOTE: All lines have only two directions except for Yellow, which has all four. We will correct this below.

array(['RedEB', 'RedWB', 'OrangeEB', 'OrangeWB', 'SilverEB', 'SilverWB',
       'BlueEB', 'BlueWB', 'GreenNB', 'GreenSB', 'YellowNB', 'YellowEB',
       'YellowSB', 'YellowWB'], dtype=object)

In [22]:
# Change Yellow Line eastbound links to northbound

link_lengths.loc[(link_lengths.Line == 'Yellow') & (link_lengths.Direction == 'EB'), 'Direction'] = 'NB'

In [23]:
# Change Yellow Line westbound links to southbound

link_lengths.loc[(link_lengths.Line == 'Yellow') & (link_lengths.Direction == 'WB'), 'Direction'] = 'SB'

In [24]:
# The build_path function uses the link data for a line and direction to build an ordered list of links for that line
#    and direction.
# After the function is defined, we also build the first northbound Yellow path (Huntington to Greenbelt).

def build_path(line, start, end, direction):
    finished = False
    new_data = link_lengths[(link_lengths.Line == line) & (link_lengths.Direction == direction) & (link_lengths.StnName1 == start)]
    while not finished:
        Station1 = new_data.iloc[-1].StnName2
        new_row = link_lengths[(link_lengths.Line == line) & (link_lengths.StnName1 == Station1) & (link_lengths.Direction == direction)]
        new_stn2 = new_row['StnName2'].to_string(index=False)
        new_data = new_data.append(new_row)
        if new_stn2 == end:
            finished = True
    new_data.index = np.arange(0, len(new_data))
    return new_data

yellow_nb1 = build_path(line='Yellow', start='Huntington', end='Greenbelt', direction='NB')


In [25]:
# Build the second northbound Yellow path (Franconia-Springfield to Greenbelt).

yellow_nb2 = build_path(line='Yellow', start='Franconia-Springfield', end='Greenbelt', direction='NB')
yellow_nb2['Line'] = 'Yello+'

In [26]:
# The Green Line data includes an erroneous link from Suitland to Huntington. Delete this link.

link_lengths.loc[(link_lengths.Line=='Green') & (link_lengths.Direction=='SB')]
link_lengths = link_lengths[(link_lengths.StnName1!='Suitland') | (link_lengths.StnName2!='Huntington')]
link_lengths.loc[(link_lengths.Line=='Green') & (link_lengths.Direction=='SB')]

,StnName1,StnName2,StnCode1,StnCode2,Direction,Length,Line
4157,Greenbelt,College Park-U of Md,E10,E09,SB,12884,Green
4158,College Park-U of Md,Prince George's Plaza,E09,E08,SB,10312,Green
4159,Prince George's Plaza,West Hyattsville,E08,E07,SB,6932,Green
4160,West Hyattsville,Fort Totten,E07,E06,SB,9925,Green
4161,Fort Totten,Georgia Ave-Petworth,E06,E05,SB,8581,Green
4162,Georgia Ave-Petworth,Columbia Heights,E05,E04,SB,4780,Green
4163,Columbia Heights,U Street/African-Amer Civil War Memorial,E04,E03,SB,4734,Green
4164,U Street/African-Amer Civil War Memorial,Shaw-Howard U,E03,E02,SB,2576,Green
4165,Shaw-Howard U,Mt Vernon Sq 7Th St-Convention Center,E02,E01,SB,2954,Green
4166,Mt Vernon Sq 7Th St-Convention Center,Gallery Pl-Chinatown,E01,F01,SB,2579,Green


In [27]:
# The Yellow Line has two southern branches. When we build each southbound path, we have to delete one of the branches first;
#   otherwise, the build_path function will not reliably terminate.
# Here, we delete the Franconia-Springfield branch before building the Greenbelt to Huntington southbound path.

ll_copy = link_lengths.copy()
link_lengths = link_lengths[(link_lengths.Line != 'Yellow') | ((link_lengths.StnName2 != 'Van Dorn Street') & (link_lengths.StnName2 != 'Franconia-SpringField'))]
yellow_sb1 = build_path(line='Yellow', start='Greenbelt', end='Huntington', direction='SB')
link_lengths = ll_copy

In [28]:
# And here (see previous cell), we delete the Huntington branch before building the Greenbelt to Franconia-Springfield path.

ll_copy = link_lengths.copy()
link_lengths = link_lengths[(link_lengths.Line != 'Yellow') | ((link_lengths.StnName2 != 'Eisenhower Avenue') & (link_lengths.StnName2 != 'Huntington'))]
yellow_sb2 = build_path(line='Yellow', start='Greenbelt', end='Franconia-Springfield', direction='SB')
yellow_sb2['Line'] = 'Yello+'
link_lengths = ll_copy

In [29]:
# Build the rest of the paths for all lines. I separated these into individual cells to make it easier to verify that each one
#   was running successfully, or to know which one failed if there was a failure (usually seen in the form of non-termination).

red_eb = build_path(line='Red', start='Shady Grove', end='Glenmont', direction='EB')

In [30]:
red_wb = build_path(line='Red', start='Glenmont', end='Shady Grove', direction='WB')

In [31]:
blue_eb = build_path(line='Blue', start='Franconia-Springfield', end='Largo Town Center', direction='EB')

In [32]:
blue_wb = build_path(line='Blue', start='Largo Town Center', end='Franconia-Springfield', direction='WB')

In [33]:
green_nb = build_path(line='Green', start='Branch Ave', end='Greenbelt', direction='NB')

In [34]:
green_sb = build_path(line='Green', start='Greenbelt', end='Branch Ave', direction='SB')

In [35]:
orange_eb = build_path(line='Orange', start='Vienna/Fairfax-GMU', end='New Carrollton', direction='EB')

In [36]:
orange_wb = build_path(line='Orange', start='New Carrollton', end='Vienna/Fairfax-GMU', direction='WB')

In [37]:
silver_eb = build_path(line='Silver', start='Wiehle-Reston East', end='Largo Town Center', direction='EB')

In [38]:
silver_wb = build_path(line='Silver', start='Largo Town Center', end='Wiehle-Reston East', direction='WB')

In [39]:
# Function that creates all trips between any two stations for a single line and direction

def single_trips(indata):
    outdata = pd.DataFrame(columns=['Line', 'StnName1', 'StnName2', 'Length', 'Transfer'])
    for i in np.arange(0, indata.index[-1]+1):
        for j in np.arange(i, indata.index[-1]+1):
            sub_data = indata[i:j+1]
#            print(sub_data)
#            print(sub_data.iloc[0,0])
            StnName1 = sub_data.iloc[0,0]
            StnName2 = sub_data.iloc[-1,1]
            Line = sub_data.iloc[0,6]
            Length = sub_data.Length.sum()
            Transfer = 0 # This is a single-line trip
            outdata = outdata.append([{'Line':Line, 'StnName1':StnName1, 'StnName2':StnName2, 'Length':Length, 'Transfer':Transfer}])
    return outdata

# Create all trips for each line and direction

green_nb_ts = single_trips(green_nb)
green_sb_ts = single_trips(green_sb)
yellow_nb1_ts = single_trips(yellow_nb1)
yellow_sb1_ts = single_trips(yellow_sb1)
yellow_nb2_ts = single_trips(yellow_nb2)
yellow_sb2_ts = single_trips(yellow_sb2)
blue_eb_ts = single_trips(blue_eb)
blue_wb_ts = single_trips(blue_wb)
orange_eb_ts = single_trips(orange_eb)
orange_wb_ts = single_trips(orange_wb)
silver_eb_ts = single_trips(silver_eb)
silver_wb_ts = single_trips(silver_wb)
red_eb_ts = single_trips(red_eb)
red_wb_ts = single_trips(red_wb)


In [40]:
# Combine all single trips

all_single = green_nb_ts.append(green_sb_ts).append(yellow_nb1_ts).append(yellow_sb1_ts).append(yellow_nb2_ts). \
             append(yellow_sb2_ts).append(blue_eb_ts).append(blue_wb_ts).append(orange_eb_ts).append(orange_wb_ts). \
             append(silver_eb_ts).append(silver_wb_ts).append(red_eb_ts).append(red_wb_ts)
all_single

,Line,StnName1,StnName2,Length,Transfer
0,Green,Branch Ave,Suitland,8879,0
0,Green,Branch Ave,Naylor Road,16786,0
0,Green,Branch Ave,Southern Avenue,23425,0
0,Green,Branch Ave,Congress Heights,29065,0
0,Green,Branch Ave,Anacostia,35804,0
0,Green,Branch Ave,Navy Yard-Ballpark,41890,0
0,Green,Branch Ave,Waterfront,45426,0
0,Green,Branch Ave,L'Enfant Plaza,49578,0
0,Green,Branch Ave,Archives-Navy Memorial-Penn Quarter,52641,0
0,Green,Branch Ave,Gallery Pl-Chinatown,54519,0


In [41]:
# Specify transfer stations

xfer_stns = ['Metro Center', 'Gallery Pl-Chinatown', "L'Enfant Plaza", 'Pentagon', 'Rosslyn', 'King St-Old Town',
             'Fort Totten', 'Stadium-Armory', 'East Falls Church']

# Create new data frames for all trips that start or end at a transfer station

xfer_left = all_single.loc[all_single['StnName1'].isin(xfer_stns)]
xfer_left=xfer_left.rename(columns = {'Length':'Length1'})
xfer_right = all_single.loc[all_single['StnName2'].isin(xfer_stns)]
xfer_right=xfer_right.rename(columns = {'Length':'Length2'})


In [42]:
# Merge the two sets of transfer segments on the transfer stations to get transfer trips

xfer_all = pd.merge(xfer_right, xfer_left, left_on='StnName2', right_on='StnName1')
xfer_all

,Line_x,StnName1_x,StnName2_x,Length2,Transfer_x,Line_y,StnName1_y,StnName2_y,Length1,Transfer_y
0,Green,Branch Ave,L'Enfant Plaza,49578,0,Green,L'Enfant Plaza,Archives-Navy Memorial-Penn Quarter,3063,0
1,Green,Branch Ave,L'Enfant Plaza,49578,0,Green,L'Enfant Plaza,Gallery Pl-Chinatown,4941,0
2,Green,Branch Ave,L'Enfant Plaza,49578,0,Green,L'Enfant Plaza,Mt Vernon Sq 7Th St-Convention Center,7520,0
3,Green,Branch Ave,L'Enfant Plaza,49578,0,Green,L'Enfant Plaza,Shaw-Howard U,10474,0
4,Green,Branch Ave,L'Enfant Plaza,49578,0,Green,L'Enfant Plaza,U Street/African-Amer Civil War Memorial,13050,0
5,Green,Branch Ave,L'Enfant Plaza,49578,0,Green,L'Enfant Plaza,Columbia Heights,17784,0
6,Green,Branch Ave,L'Enfant Plaza,49578,0,Green,L'Enfant Plaza,Georgia Ave-Petworth,22564,0
7,Green,Branch Ave,L'Enfant Plaza,49578,0,Green,L'Enfant Plaza,Fort Totten,31145,0
8,Green,Branch Ave,L'Enfant Plaza,49578,0,Green,L'Enfant Plaza,West Hyattsville,41070,0
9,Green,Branch Ave,L'Enfant Plaza,49578,0,Green,L'Enfant Plaza,Prince George's Plaza,48002,0


In [43]:
# Drop all "transfer" trips where the lines are the same color, or where the start and end stations are the same; then reindex

xfer_all = xfer_all.loc[(xfer_all.Line_x != xfer_all.Line_y) & (xfer_all.StnName1_x != xfer_all.StnName2_y)]
xfer_all.index = np.arange(0, len(xfer_all))

In [44]:
# Calculate total length for each transfer trip, and set Transfer flag to 1 for all trips

Length = xfer_all.Length1 + xfer_all.Length2
Length.index = np.arange(0, len(Length))

xfer_all['Length'] = Length
xfer_all['Transfer'] = 1
xfer_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49360 entries, 0 to 49359
Data columns (total 12 columns):
Line_x        49360 non-null object
StnName1_x    49360 non-null object
StnName2_x    49360 non-null object
Length2       49360 non-null object
Transfer_x    49360 non-null object
Line_y        49360 non-null object
StnName1_y    49360 non-null object
StnName2_y    49360 non-null object
Length1       49360 non-null object
Transfer_y    49360 non-null object
Length        49360 non-null object
Transfer      49360 non-null int64
dtypes: int64(1), object(11)
memory usage: 4.9+ MB


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [45]:
# Drop unnecessary columns

xfer_all.drop(['StnName2_x', 'Length1', 'Transfer_x', 'StnName1_y', 'Length2', 'Transfer_y'], axis='columns', inplace=True)
xfer_all

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Line_x,StnName1_x,Line_y,StnName2_y,Length,Transfer
0,Green,Branch Ave,Yellow,Archives-Navy Memorial-Penn Quarter,52641,1
1,Green,Branch Ave,Yellow,Gallery Pl-Chinatown,54519,1
2,Green,Branch Ave,Yellow,Mt Vernon Sq 7Th St-Convention Center,57098,1
3,Green,Branch Ave,Yellow,Shaw-Howard U,60052,1
4,Green,Branch Ave,Yellow,U Street/African-Amer Civil War Memorial,62628,1
5,Green,Branch Ave,Yellow,Columbia Heights,67362,1
6,Green,Branch Ave,Yellow,Georgia Ave-Petworth,72142,1
7,Green,Branch Ave,Yellow,Fort Totten,80723,1
8,Green,Branch Ave,Yellow,West Hyattsville,90648,1
9,Green,Branch Ave,Yellow,Prince George's Plaza,97580,1


In [46]:
# Rename most of the remaining columns

xfer_all=xfer_all.rename(columns = {'Line_x':'Line1', 'StnName1_x':'StnName1', 'Line_y':'Line2', 'StnName2_y':'StnName2'})

In [47]:
# Get shortest possible transfer trip for each station pair

xfer_min = xfer_all.iloc[xfer_all.groupby(['StnName1', 'StnName2']).apply(lambda x: x['Length'].idxmin())]

In [48]:
# Combine single trips and transfer trips

trips_all = xfer_min.append(all_single)
trips_all

,Length,Line,Line1,Line2,StnName1,StnName2,Transfer
8982,54557,NaN,Blue,Green,Addison Road-Seat Pleasant,Anacostia,1
8968,43846,NaN,Blue,Green,Addison Road-Seat Pleasant,Archives-Navy Memorial-Penn Quarter,1
14811,66171,NaN,Silver,Blue,Addison Road-Seat Pleasant,Arlington Cemetery,1
9048,75077,NaN,Blue,Orange,Addison Road-Seat Pleasant,Ballston-MU,1
45277,40179,NaN,Blue,Silver,Addison Road-Seat Pleasant,Benning Road,1
37294,86726,NaN,Blue,Red,Addison Road-Seat Pleasant,Bethesda,1
9004,78960,NaN,Blue,Yellow,Addison Road-Seat Pleasant,Braddock Road,1
8987,90361,NaN,Blue,Green,Addison Road-Seat Pleasant,Branch Ave,1
37280,67712,NaN,Blue,Red,Addison Road-Seat Pleasant,Brookland-CUA,1
45278,47524,NaN,Blue,Silver,Addison Road-Seat Pleasant,Capitol Heights,1


In [49]:
# Get shortest trip for each station pair as follows:
# 1. Sort all trips by station 1, station 2, length, transfer
# 2. Use the first() function to select the trip to keep. This ensures that, if we have equally long trips where one requires a
#    transfer and one doesn't, we select the trip that doesn't require a transfer.

trips_min = trips_all.sort_values(by=['StnName1', 'StnName2', 'Length', 'Transfer']).groupby(['StnName1', 'StnName2'], sort=False, as_index=False).first()
trips_min.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8190 entries, 0 to 8189
Data columns (total 7 columns):
StnName1    8190 non-null object
StnName2    8190 non-null object
Length      8190 non-null int64
Line        2886 non-null object
Line1       7456 non-null object
Line2       7456 non-null object
Transfer    8190 non-null int64
dtypes: int64(2), object(5)
memory usage: 511.9+ KB


In [50]:
# The vol_data (volume data, read in at the beginning of this notebook) station names and the trips_min station names (from
# station_data) don't match. Get a list of the trips_min station names to prepare for renaming these to match.

trip_stns = pd.DataFrame(trips_min.StnName1.unique())
trip_stns.sort_values(by=0, inplace=True)
trip_stns

,0
0,Addison Road-Seat Pleasant
1,Anacostia
2,Archives-Navy Memorial-Penn Quarter
3,Arlington Cemetery
4,Ballston-MU
5,Benning Road
6,Bethesda
7,Braddock Road
8,Branch Ave
9,Brookland-CUA


In [51]:
# Get a list of the vol_data station names.

vol_stns = pd.DataFrame(vol_data.ENTSTATION.unique())
vol_stns.sort_values(by=0, inplace=True)
vol_stns

,0
38,Addison Road
47,Anacostia
9,Archives-Navy Memorial
63,Arlington Cemetery
83,Ballston
29,Benning Road
14,Bethesda
42,Braddock Road
86,Branch Avenue
25,Brookland


In [52]:
# Merge the two sets of station names to see which ones don't match.

both_stns = pd.merge(trip_stns, vol_stns, on=0, how='outer', indicator=True).sort_values(by=0)
both_stns

,0,_merge
91,Addison Road,right_only
0,Addison Road-Seat Pleasant,left_only
1,Anacostia,both
92,Archives-Navy Memorial,right_only
2,Archives-Navy Memorial-Penn Quarter,left_only
3,Arlington Cemetery,both
93,Ballston,right_only
4,Ballston-MU,left_only
5,Benning Road,both
6,Bethesda,both


In [53]:
# Get the station names that don't match.
# "right_only" are the vol_data names. We'll use these as the new names for trips_min.
# "left_only" are the existing trips_min names.

new_val = pd.DataFrame(both_stns.loc[both_stns._merge == 'right_only'][0]).sort_values(by=0)
old_val = pd.DataFrame(both_stns.loc[both_stns._merge == 'left_only'][0]).sort_values(by=0)

In [54]:
# Create dictionary of station name pairs for renaming.

stn_rename = dict(zip(old_val[0], new_val[0]))

In [55]:
stn_rename

{'Addison Road-Seat Pleasant': 'Addison Road',
 'Archives-Navy Memorial-Penn Quarter': 'Archives-Navy Memorial',
 'Ballston-MU': 'Ballston',
 'Branch Ave': 'Branch Avenue',
 'Brookland-CUA': 'Brookland',
 'College Park-U of Md': 'College Park-U of MD',
 'Dunn Loring-Merrifield': 'Dunn Loring',
 'Foggy Bottom-GWU': 'Foggy Bottom',
 'Gallery Pl-Chinatown': 'Gallery Place-Chinatown',
 'Georgia Ave-Petworth': 'Georgia Avenue-Petworth',
 'Grosvenor-Strathmore': 'Grosvenor',
 'King St-Old Town': 'King Street',
 'Minnesota Ave': 'Minnesota Avenue',
 'Morgan Boulevard': 'Morgan Blvd.',
 'Mt Vernon Sq 7Th St-Convention Center': 'Mt. Vernon Square-UDC',
 'Navy Yard-Ballpark': 'Navy Yard',
 'Noma-Gallaudet': 'New York Ave',
 'Potomac Ave': 'Potomac Avenue',
 'Rhode Island Ave-Brentwood': 'Reagan Washington National Airport',
 'Ronald Reagan Washington National Airpor': 'Rhode Island Avenue',
 'Shaw-Howard U': 'Shaw-Howard University',
 'U Street/African-Amer Civil War Memorial': 'U Street-Cardozo

In [57]:
# Use the dictionary to replace the names on trips_min.

trips_min.replace({'StnName1': stn_rename, 'StnName2': stn_rename}, inplace=True)
trips_min

,StnName1,StnName2,Length,Line,Line1,Line2,Transfer
0,Addison Road,Anacostia,54557,NaN,Blue,Green,1
1,Addison Road,Archives-Navy Memorial,43846,NaN,Blue,Green,1
2,Addison Road,Arlington Cemetery,66171,Blue,Silver,Blue,0
3,Addison Road,Ballston,75077,Silver,Blue,Orange,0
4,Addison Road,Benning Road,12891,Blue,Blue,Silver,0
5,Addison Road,Bethesda,86726,NaN,Blue,Red,1
6,Addison Road,Braddock Road,78960,Blue,Blue,Yellow,1
7,Addison Road,Branch Avenue,90361,NaN,Blue,Green,1
8,Addison Road,Brookland,67712,NaN,Blue,Red,1
9,Addison Road,Capitol Heights,5546,Blue,Blue,Silver,0


In [58]:
# Merge volume data and trips data (lengths and transfer flag) for analysis.
# NOTE: Volume data includes trips that start and end at the same station. Most of these are probably "give-ups," where the
#       passenger gave up before boarding a train, and the trip lengths are zero, so we do an inner join to exclude these.
#       They were already excluded from the trips data.

merged_data = pd.merge(vol_data, trips_min, left_on=['ENTSTATION', 'EXTSTATION'], right_on=['StnName1', 'StnName2'], 
                       sort=True, how='inner')
merged_data

,ENTDATESERVICETYPE,ENTDATEDAYOFWEEK,ENTSTATION,EXTSTATION,ENTPERIOD,ENTQUARTHOUR,AVG_TRIPS,StnName1,StnName2,Length,Line,Line1,Line2,Transfer
0,Weekday,Wed,Addison Road,Anacostia,Evening,9:00 PM to 9:15 PM,2,Addison Road,Anacostia,54557,NaN,Blue,Green,1
1,Weekday,Wed,Addison Road,Anacostia,Evening,9:30 PM to 9:45 PM,2,Addison Road,Anacostia,54557,NaN,Blue,Green,1
2,Weekday,Thu,Addison Road,Anacostia,AM Peak,5:30 AM to 5:45 AM,1,Addison Road,Anacostia,54557,NaN,Blue,Green,1
3,Weekday,Wed,Addison Road,Anacostia,PM Peak,4:30 PM to 4:45 PM,1,Addison Road,Anacostia,54557,NaN,Blue,Green,1
4,Weekday,Fri,Addison Road,Anacostia,AM Peak,6:45 AM to 7:00 AM,1,Addison Road,Anacostia,54557,NaN,Blue,Green,1
5,Weekday,Mon,Addison Road,Anacostia,PM Peak,5:15 PM to 5:30 PM,1,Addison Road,Anacostia,54557,NaN,Blue,Green,1
6,Saturday,Sat,Addison Road,Anacostia,AM Peak,6:45 AM to 7:00 AM,1,Addison Road,Anacostia,54557,NaN,Blue,Green,1
7,Weekday,Tue,Addison Road,Anacostia,AM Peak,6:15 AM to 6:30 AM,1,Addison Road,Anacostia,54557,NaN,Blue,Green,1
8,Sunday,Sun,Addison Road,Anacostia,PM Peak,6:15 PM to 6:30 PM,1,Addison Road,Anacostia,54557,NaN,Blue,Green,1
9,Weekday,Fri,Addison Road,Anacostia,AM Peak,6:00 AM to 6:15 AM,1,Addison Road,Anacostia,54557,NaN,Blue,Green,1


In [59]:
type_dict = {'ENTDATESERVICETYPE':'str', 'ENTDATEDAYOFWEEK':'str', 'ENTSTATION':'str', 'EXTSTATION':'str', 'ENTPERIOD':'str',
             'ENTQUARTHOUR':'str', 'StnName1':'str', 'StnName2':'str', 'Line':'str', 'Line1':'str', 'Line2':'str'}

merged_data.to_sql('merged_data', con=conn, index=False, dtype=type_dict, if_exists='replace')